# Case ID Assignment Project

### Imports


In [1]:
import numpy as np
import pandas as pd
from sklearn.pipeline import Pipeline

import case_id_assignment.feature_engineering as features_eng
import case_id_assignment.feature_selection as selector
import case_id_assignment.clustering as clustering
import case_id_assignment.utilities as util
import case_id_assignment.imputing as imputer
import case_id_assignment.sqlutil as sql
import case_id_assignment.assignment as case_id_assigner
import case_id_assignment.evaluation as evaluation
import case_id_assignment.main as main

## 1) Loading the data

In [2]:
data_folder = '../data'
isolated_data_set = util.load_data_set(file_path=f'{data_folder}/ptp_isolated_data.csv')
interleaved_data_set = util.load_data_set(file_path=f'{data_folder}/ptp_interleaved_data.csv')

After loading the data we can observe its structure. We can notice the difference between the isolated and the interleaved data. In the interleaved data, the  BusinessActivity and InstanceNumber
contains garbage data while in the isolated data they contain the actual BusinessActivity and InstanceNumber

In [3]:
isolated_data_set.head()

,FileName,BusinessActivity,InstanceNumber,sniff_time,frame.number,synthetic_sniff_time,synthetic_sniff_time_str,session_generalized,HighestLayerProtocol,MessageType_WithRole,MessageType,MessageAttributes,query_type,session_class,filter_flag,query,tables,event,event_with_roles,noise_event
0,CreatePurchaseRequest_1.pcap,CreatePurchaseRequest,1,2020-12-21 08:08:44.637118,119,2020-12-22 11:39:44.237118,2020-12-22 11:39:44.237118,192.168.11.2-192.168.11.9 (1),http,End Point (Employee)->Odoo Application:[HttpRe...,HttpRequest:POST /xmlrpc/2/common HTTP/1.1\r\n,"{'': 'POST /xmlrpc/2/common HTTP/1.1\\r\\n', '...",NaN,http,True,NaN,NaN,HttpRequest:POST /xmlrpc/2/common HTTP/1.1\r\n,End Point (Employee)->Odoo Application: [HttpR...,False
1,CreatePurchaseRequest_1.pcap,CreatePurchaseRequest,1,2020-12-21 08:08:44.688344,153,2020-12-22 11:48:16.548344,2020-12-22 11:48:16.548344,192.168.11.2-192.168.11.9 (1),http,Odoo Application->End Point (Employee):[HttpRe...,HttpResponse:HTTP/1.0 200 OK\r\n,"{'': 'HTTP/1.0 200 OK\\r\\n', '_ws.expert': 'E...",NaN,http,True,NaN,NaN,HttpResponse:HTTP/1.0 200 OK\r\n,Odoo Application->End Point (Employee): [HttpR...,False
2,CreatePurchaseRequest_1.pcap,CreatePurchaseRequest,1,2020-12-21 08:08:44.690577,161,2020-12-22 11:48:38.880577,2020-12-22 11:48:38.880577,192.168.11.2-192.168.11.9 (2),http,End Point (Employee)->Odoo Application:[HttpRe...,HttpRequest:POST /xmlrpc/2/common HTTP/1.1\r\n,"{'': 'POST /xmlrpc/2/common HTTP/1.1\\r\\n', '...",NaN,http,True,NaN,NaN,HttpRequest:POST /xmlrpc/2/common HTTP/1.1\r\n,End Point (Employee)->Odoo Application: [HttpR...,False
3,CreatePurchaseRequest_1.pcap,CreatePurchaseRequest,1,2020-12-21 08:08:44.985813,224,2020-12-22 12:37:51.535813,2020-12-22 12:37:51.535813,192.168.11.1-192.168.11.2 (7),pgsql,Odoo Application->db Server/Mail Server:[Pgsql...,PgsqlRequest:Simple query,"{'pgsql.type': 'Simple query', 'pgsql.length':...",INSERT,pgsql,True,"INSERT INTO ""res_users_log"" (""id"", ""create_uid...",['res_users_log'],PgsqlRequest:Simple query:INSERT:['res_users_l...,Odoo Application->db Server/Mail Server: [Pgsq...,False
4,CreatePurchaseRequest_1.pcap,CreatePurchaseRequest,1,2020-12-21 08:08:44.994598,245,2020-12-22 12:39:19.394598,2020-12-22 12:39:19.394598,192.168.11.2-192.168.11.9 (2),http,Odoo Application->End Point (Employee):[HttpRe...,HttpResponse:HTTP/1.0 200 OK\r\n,"{'': 'HTTP/1.0 200 OK\\r\\n', '_ws.expert': 'E...",NaN,http,True,NaN,NaN,HttpResponse:HTTP/1.0 200 OK\r\n,Odoo Application->End Point (Employee): [HttpR...,False


In [4]:
interleaved_data_set.head()

,FileName,BusinessActivity,InstanceNumber,sniff_time,frame.number,synthetic_sniff_time,synthetic_sniff_time_str,session_generalized,HighestLayerProtocol,MessageType_WithRole,...,event,event_with_roles,noise_event,real_activity,real_activity_action,real_case_id,activities,frame_number,stream_index,activities_with_bp
0,async_2022_08_03-0336_10_cases_10_interval.pcap,order_to_cash_interleaved_10cases,2022,2022-08-03 13:36:21.687676,96,2022-08-04 21:07:16.297676,2022-08-04 21:07:16.297676,192.168.11.2-192.168.11.10 (1),http,End Point (Employee)->Odoo Application:[HttpRe...,...,HttpRequest:POST /xmlrpc/2/common HTTP/1.1\r\n,End Point (Employee)->Odoo Application: [HttpR...,False,CreatePurchaseRequest Start,Activity Start,399.0,['CreatePurchaseRequest'],96,2,"[(-1, ""CreatePurchaseRequest"")]"
1,async_2022_08_03-0336_10_cases_10_interval.pcap,order_to_cash_interleaved_10cases,2022,2022-08-03 13:36:21.958633,137,2022-08-04 21:52:26.138633,2022-08-04 21:52:26.138633,192.168.11.2-192.168.11.10 (1),http,Odoo Application->End Point (Employee):[HttpRe...,...,HttpResponse:HTTP/1.0 200 OK\r\n,Odoo Application->End Point (Employee): [HttpR...,False,NoAction,NoAction,NaN,['CreatePurchaseRequest'],137,2,"[(-1, ""CreatePurchaseRequest"")]"
2,async_2022_08_03-0336_10_cases_10_interval.pcap,order_to_cash_interleaved_10cases,2022,2022-08-03 13:36:21.960536,145,2022-08-04 21:52:45.170536,2022-08-04 21:52:45.170536,192.168.11.2-192.168.11.10 (2),http,End Point (Employee)->Odoo Application:[HttpRe...,...,HttpRequest:POST /xmlrpc/2/common HTTP/1.1\r\n,End Point (Employee)->Odoo Application: [HttpR...,False,NoAction,NoAction,NaN,['CreatePurchaseRequest'],145,7,"[(-1, ""CreatePurchaseRequest"")]"
3,async_2022_08_03-0336_10_cases_10_interval.pcap,order_to_cash_interleaved_10cases,2022,2022-08-03 13:36:22.298610,209,2022-08-04 22:49:06.248610,2022-08-04 22:49:06.248610,192.168.11.1-192.168.11.2 (7),pgsql,Odoo Application->db Server/Mail Server:[Pgsql...,...,PgsqlRequest:Simple query:INSERT:['res_users_l...,Odoo Application->db Server/Mail Server: [Pgsq...,False,NoAction,NoAction,NaN,['CreatePurchaseRequest'],209,10,"[(-1, ""CreatePurchaseRequest"")]"
4,async_2022_08_03-0336_10_cases_10_interval.pcap,order_to_cash_interleaved_10cases,2022,2022-08-03 13:36:22.413521,232,2022-08-04 23:08:15.473521,2022-08-04 23:08:15.473521,192.168.11.2-192.168.11.10 (2),http,Odoo Application->End Point (Employee):[HttpRe...,...,HttpResponse:HTTP/1.0 200 OK\r\n,Odoo Application->End Point (Employee): [HttpR...,False,NoAction,NoAction,NaN,['CreatePurchaseRequest'],232,7,"[(-1, ""CreatePurchaseRequest"")]"


We can observe the structure of the data sets. Most columns are objects (str). Before pre-processing and feature engineering we have 20 and 27 columns for isolated and interleaved data set respectively.
In the data exploration step we noticed no column is correlated with the InstanceNumber column. So we turn to pre-processing and feature engineering step.

In [5]:
isolated_data_set.info()

<class 'pandas.core.frame.DataFrame'>
Index: 26006 entries, 0 to 26005
Data columns (total 20 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   FileName                  26006 non-null  object
 1   BusinessActivity          26006 non-null  object
 2   InstanceNumber            26006 non-null  int64 
 3   sniff_time                26006 non-null  object
 4   frame.number              26006 non-null  int64 
 5   synthetic_sniff_time      26006 non-null  object
 6   synthetic_sniff_time_str  26006 non-null  object
 7   session_generalized       26006 non-null  object
 8   HighestLayerProtocol      26006 non-null  object
 9   MessageType_WithRole      26006 non-null  object
 10  MessageType               26006 non-null  object
 11  MessageAttributes         26006 non-null  object
 12  query_type                20503 non-null  object
 13  session_class             26006 non-null  object
 14  filter_flag               2

In [6]:
interleaved_data_set.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3717 entries, 0 to 3716
Data columns (total 27 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   FileName                  3717 non-null   object 
 1   BusinessActivity          3717 non-null   object 
 2   InstanceNumber            3717 non-null   int64  
 3   sniff_time                3717 non-null   object 
 4   frame.number              3717 non-null   int64  
 5   synthetic_sniff_time      3717 non-null   object 
 6   synthetic_sniff_time_str  3717 non-null   object 
 7   session_generalized       3717 non-null   object 
 8   HighestLayerProtocol      3717 non-null   object 
 9   MessageType_WithRole      3717 non-null   object 
 10  MessageType               3717 non-null   object 
 11  MessageAttributes         3717 non-null   object 
 12  query_type                2914 non-null   object 
 13  session_class             3717 non-null   object 
 14  filter_flag  

## 2) Pre-processing
In this step we are going to pre-process the data. The main step here is creating features from the sql queries in the query, MessageAttributes and tables columns

In [7]:
isolated_df_processed, interleaved_df_processed = main.pre_processing_data(isolated_data_set, interleaved_data_set,
                                                                           save_results=False,
                                                                           pre_process=True)

Process isolated data set - parse SQL queries to features


Convert SQL Queries into additional columns:   0%|          | 0/26006 [00:00<?, ?it/s]

  0%|          | 0/26006 [00:00<?, ?it/s]

finished processing the data, create data frames
merge data frames to one global data frame
Process interleaved data set - parse SQL queries to features


Convert SQL Queries into additional columns:   0%|          | 0/3717 [00:00<?, ?it/s]

  0%|          | 0/3717 [00:00<?, ?it/s]

finished processing the data, create data frames
merge data frames to one global data frame
Process isolated data set - HTTP attributes to features
Process interleaved data set - HTTP attributes to features


After running the pre-processing step we can observe the new features
We can see additional features such as warehouse_id, res_model, sequence, sale_order_id etc. were created.
Most of the data contain Nan so the data set is a sparse data set.
We can see from the info() we now have 239 columns.

In [8]:
isolated_df_processed.head()

,FileName,BusinessActivity,InstanceNumber,sniff_time,frame.number,synthetic_sniff_time,synthetic_sniff_time_str,session_generalized,HighestLayerProtocol,MessageType_WithRole,...,multi,payment_date,payment_difference_handling,payment_method_id,payment_type,writeoff_label,payment_term_id,request_method_call,starting_frame_number,file_data
0,CreatePurchaseRequest_1.pcap,CreatePurchaseRequest,1,2020-12-21 08:08:44.637118,119,2020-12-22 11:39:44.237118,2020-12-22 11:39:44.237118,192.168.11.2-192.168.11.9 (1),http,End Point (Employee)->Odoo Application:[HttpRe...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,version,NaN,[version]
1,CreatePurchaseRequest_1.pcap,CreatePurchaseRequest,1,2020-12-21 08:08:44.688344,153,2020-12-22 11:48:16.548344,2020-12-22 11:48:16.548344,192.168.11.2-192.168.11.9 (1),http,Odoo Application->End Point (Employee):[HttpRe...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,server_version,119,"[server_version, 12.0-20190820, server_version..."
2,CreatePurchaseRequest_1.pcap,CreatePurchaseRequest,1,2020-12-21 08:08:44.690577,161,2020-12-22 11:48:38.880577,2020-12-22 11:48:38.880577,192.168.11.2-192.168.11.9 (2),http,End Point (Employee)->Odoo Application:[HttpRe...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,authenticate,NaN,"[authenticate, odoo01, user.suername@company.c..."
3,CreatePurchaseRequest_1.pcap,CreatePurchaseRequest,1,2020-12-21 08:08:44.985813,224,2020-12-22 12:37:51.535813,2020-12-22 12:37:51.535813,192.168.11.1-192.168.11.2 (7),pgsql,Odoo Application->db Server/Mail Server:[Pgsql...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,[]
4,CreatePurchaseRequest_1.pcap,CreatePurchaseRequest,1,2020-12-21 08:08:44.994598,245,2020-12-22 12:39:19.394598,2020-12-22 12:39:19.394598,192.168.11.2-192.168.11.9 (2),http,Odoo Application->End Point (Employee):[HttpRe...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,161,[2]


In [9]:
isolated_df_processed.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26006 entries, 0 to 26005
Columns: 239 entries, FileName to file_data
dtypes: bool(2), float64(24), int64(2), object(211)
memory usage: 47.1+ MB


## 3) Feature Engineering

In this step we create additional features from the new columns created in the pre-processing step. We break down some columns to additional columns, we create new columns from the tables' column, and we clean some values.
After running this step we can see we now have 258 columns.

In [10]:
# ---- Feature Engineering, manipulate features, change format, process value etc --- #
isolated_df_engineered, interleaved_df_engineered = main.engineer_features(isolated_df_processed,
                                                                           interleaved_df_processed,
                                                                           feature_engineering=True)

Impute values from parse_table_column: 100%|██████████| 1867/1867 [00:03<00:00, 587.56it/s] 


In [11]:
isolated_df_engineered.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26006 entries, 0 to 26005
Columns: 258 entries, FileName to account_move_id
dtypes: bool(2), float64(38), int64(2), object(216)
memory usage: 50.8+ MB


## 4) Feature Selection
In this step we select features for the clustering. We choose the features from teh isolated data set since we have the InstanceNumber column.
We check the correlation of between all columns and Instance number, and we select all columns correlated with Instance number above 0.90


In [12]:
list_of_features = selector.simple_correlation_selector(isolated_df_engineered, target_column='InstanceNumber',
                                                        save_to_file=False,
                                                        threshold=0.90)

print(f'Selected Features:{list_of_features}')

Selected Features:['date_order', 'mail_followers_id', 'date_planned', 'parent_id', 'scheduled_date', 'res_name', 'attachment_id', 'sale_order_id', 'group_id', 'origin', 'new_value_datetime', 'date_expected', 'purchase_line_id', 'purchase_order_id', 'date_done', 'move_name', 'number', 'invoice_id', 'ref', 'sale_order_line_id', 'purchase_requisition_line_id', 'purchase_requisition_id', 'ir_attachment_id', 'purchase_order_line_id', 'stock_move_id', 'stock_move_line_id', 'account_invoice_line_id', 'account_move_line_id', 'account_move_id']


## 5) Data imputing
In this step we impute the data in the interleaved data set.
The idea is to fill in attributes values, so we can use them to assign a case id based on the values.
Note - This step can take about 10 minutes to complete

In [13]:
interleaved_df_imputed = main.impute(interleaved_df_engineered, save_results=False, impute=True)

Missing data percentage before imputation:0.880924640075204


Impute values from extract: 100%|██████████| 3717/3717 [00:03<00:00, 1060.78it/s]
Collect values to imputer: extract_po_from_res_id: 100%|██████████| 3717/3717 [00:00<00:00, 6150.64it/s]
Impute values from extract_po_from_res_id: 100%|██████████| 118/118 [00:00<00:00, 1450.90it/s]
Collect values to imputer: ImputeFromRequestMethodCall: 100%|██████████| 3717/3717 [00:00<00:00, 1239089.81it/s]
C:\Research\cade-id-assignment\src\case_id_assignment\imputing.py:367: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  X['attribute_name'] = X['file_data'].progress_apply(_extract_attribute_name)
Collect values to imputer: ImputeFromRequestMethodCall: 100%|██████████| 3717/3717 [00:01<00:00, 2402.05it/s]
Impute values from _extreact_from_request_method: 100%|██████████| 3

Missing data percentage after imputation:0.7869893414687603


## 6) Clustering
Using the features detected in the previous step and the imputed data, we cluster the data using graph creation and community detection.
We experiment with 3 community detection algorithms. Here we run the greedy_modularity_communities

In [15]:
filtered_df = interleaved_df_imputed[list_of_features]
clusters = clustering.greedy_modularity_communities(data_set=filtered_df, save_to_file=False)
# clusters = clustering.girvan_newman(data_set=filtered_df)
# clusters = clustering.louvain_communities(data_set=filtered_df)

print(f'Number of clusters : {len(clusters)}')

Number of clusters : 48


In [16]:
clusters

[frozenset({('account_invoice_line_id', 256.0),
            ('account_move_id', 270.0),
            ('account_move_line_id', 544.0),
            ('account_move_line_id', 545.0),
            ('attachment_id', '6256'),
            ('attachment_id', '6257'),
            ('attachment_id', '6267'),
            ('date_done', '2022-08-03 10:17:13'),
            ('date_expected', '2022-08-03 05:37:50.209211'),
            ('date_order', '2022-08-03 10:16:26'),
            ('date_order', '2022-08-03 10:16:35'),
            ('date_planned', '2022-08-03 05:37:50.209211'),
            ('date_planned', '2022-08-03 10:16:35'),
            ('group_id', '246'),
            ('invoice_id', '252'),
            ('invoice_id', 252.0),
            ('ir_attachment_id', 6256.0),
            ('ir_attachment_id', 6257.0),
            ('ir_attachment_id', 6267.0),
            ('mail_followers_id', '6190'),
            ('mail_followers_id', '6193'),
            ('mail_followers_id', '6194'),
            ('mail_fo

## 7) Case id assignment
Now that we have the clusters we can use them to assign a case id to packets containing the features we identify in the feature selection step.
We enumerate the clusters giving each cluster a number.
In the interleaved data, we extract features' values from each row and check to which cluster these features belongs to.
The row is then assigned with the identified cluster's number as a case id.
For rows which did not have any attributes, we assign the closes packet's case id


In [17]:
results_data_set = case_id_assigner.assign_case_id(data_set=interleaved_df_imputed, attributes=list_of_features,
                                                   clusters=clusters)
results_data_set = case_id_assigner.assign_case_id_to_activity_action(data_set=interleaved_df_imputed)

0it [00:00, ?it/s]

Assign case id to packets containing attributes values:   0%|          | 0/3717 [00:00<?, ?it/s]

C:\Research\cade-id-assignment\src\case_id_assignment\assignment.py:41: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  data_set['case_id'] = case_id_column


Assign the closest case id to activity actions:   0%|          | 0/3717 [00:00<?, ?it/s]

C:\Research\cade-id-assignment\src\case_id_assignment\assignment.py:111: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  data_set['CaseIDVoting'] = case_id_voting


We can observe the case id assignment in the

In [18]:
results_data_set[['real_activity_action', 'real_case_id', 'case_id', 'CaseIDVoting']]

,real_activity_action,real_case_id,case_id,CaseIDVoting
0,Activity Start,399.0,set(),{9}
1,NoAction,399.0,set(),set()
2,NoAction,NaN,set(),set()
3,NoAction,NaN,set(),set()
4,NoAction,NaN,set(),set()
...,...,...,...,...
3712,NoAction,NaN,{36},{36}
3713,NoAction,NaN,{36},{36}
3714,NoAction,NaN,{36},{36}
3715,NoAction,NaN,{36},{36}


## 8) Evaluation
In this step we evaluate the clustering.
We use the following metrics for evaluation :
results_data_set = case_id_assigner.assign_case_id_to_activity_action(data_set=interleaved_df_imputed)
1) Rand index
2) Homogeneity, completeness and V-measure.
3)
Rand index measures the similarity of the two assignments, ignoring permutations. Homogeneity measures if
each cluster contains only members of a single class, completeness measures if all members of a given class are
assigned to the same cluster, V-measure is their harmonic mean. For all metrics, the closes to 1 the better and
vice versa.


In [20]:
rand_score, homogeneity, completeness, v_measure = evaluation.evaluate_case_id_accuracy(
    data_set=results_data_set)

print(f'Rand Score : {rand_score}')
print(f'Homogeneity : {homogeneity}')
print(f'Completeness : {completeness}')
print(f'V_measure : {v_measure}')

Rand Score : 0.8957460317460317
Homogeneity : 0.6600092899816811
Completeness : 0.4858060794105318
V_measure : 0.5596652551634177
